<a href="https://colab.research.google.com/github/DepartmentOfStatisticsPUE/ann-for-survey-sampling/blob/main/notebooks/ann_paper_sim_study_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required packages for the simulation study

In [60]:
!pip install numba
!pip install annoy
!pip install pynndescent
!pip install nmslib ## hnswlib https://github.com/nmslib/hnswlib

Load standard modules

In [1]:
import pandas as pd
import numpy as np
import plotnine as ggplot

Load ann modules

In [2]:
from scipy.spatial import KDTree ## allows for approximate search within query
from scipy.spatial import cKDTree ## allows for approximate search within query
from annoy import AnnoyIndex
import nmslib 
from pynndescent import NNDescent

Functions for the study

In [3]:
def kdtree_impute(tree, sample, data, y, x, eps = 0):
  nns = tree.query(sample[:,x], k = 1, eps = eps)
  res = np.mean(data[nns[1]][:, ys], axis = 0)
  return res

def nmslib_create_index(data, y, x, space_name='l2', M = 15, efC = 100, num_threads = 4):
  index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
  index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
  index.addDataPointBatch(data[:,x])
  index.createIndex(index_time_params) 
  return index

def nmslib_impute(index, sample, data, y, x, num_threads = 4):
  nbrs = index.knnQueryBatch(sample[:,x], k = 1, num_threads = num_threads)
  inds = [i[0][0] for i in nbrs]
  res = np.mean(data[inds][:,y], axis=0)
  return res

Generate data for the simulation study

In [35]:
np.random.seed(123)
N = 10000
x1 = np.random.normal(loc=1.0,scale=1.0,size=N)
x2 = np.random.exponential(scale=1.0, size = N)
epsilon = np.random.normal(size=N)

### target variables
y1 = 1 + x1 + x2 + epsilon
y2 = 0.5*(x1 - 1.5)**2 + x2 + epsilon
## propensity scores
p1 = np.exp(x2) / (1 + np.exp(x2))
p2 = np.exp(-0.5 + 0.5*(x2-2)**2) / (1 + np.exp(-0.5 + 0.5*(x2-2)**2))

data = np.column_stack((x1,x2,y1,y2,p1,p2))
data[:5]

array([[-0.0856306 ,  2.43687818,  3.02219813,  3.36494093,  0.91959657,
         0.40021515],
       [ 1.99734545,  2.87627775,  6.07811088,  3.20444168,  0.94666123,
         0.47101537],
       [ 1.2829785 ,  0.54269314,  2.71588158,  0.45645225,  0.63243869,
         0.63688549],
       [-0.50629471,  0.1961151 ,  2.04167524,  3.56057919,  0.54887223,
         0.7552849 ],
       [ 0.42139975,  0.90549549,  2.27648533,  1.43677483,  0.71207752,
         0.52472234]])

Simulation study

In [43]:
R = 500
xs = [0,1]
ys = [2,3]
ps = [4,5]

results_kdtree_500 = np.zeros(shape = (R, 2))
results_kdtree_1000 = np.zeros(shape = (R, 2))
results_kdtree_app_500 = np.zeros(shape = (R, 2))
results_kdtree_app_1000 = np.zeros(shape = (R, 2))
results_nmslib_500 = np.zeros(shape = (R, 2))
results_nmslib_1000 = np.zeros(shape = (R, 2))
results_nnd_500 = np.zeros(shape = (R, 2))
results_nnd_1000 = np.zeros(shape = (R, 2))
results_annoy_500 = np.zeros(shape = (R, 2))
results_annoy_1000 = np.zeros(shape = (R, 2))

for r in range(R):
  print(r)
  np.random.seed(r)
  ## big data sample
  big_p1 = np.random.binomial(n=1, p = p1, size = N)
  big_p2 = np.random.binomial(n=1, p = p2, size = N)    
  ## random samples
  s500 = np.random.choice(a = data.shape[0], size = 500, replace = False)
  s1000 = np.random.choice(a = data.shape[0], size = 1000, replace = False)
  
  kdtree = KDTree(data[big_p1==1][:, xs], leafsize = 50)
  results_kdtree_500[r, :] = kdtree_impute(kdtree, data[s500], data[big_p1==1], ys, xs)
  results_kdtree_1000[r, :] = kdtree_impute(kdtree, data[s1000], data[big_p1==1], ys, xs)
  results_kdtree_app_500[r, :] = kdtree_impute(kdtree, data[s500], data[big_p1==1], ys, xs, eps = 10)
  results_kdtree_app_1000[r, :] = kdtree_impute(kdtree, data[s1000], data[big_p1==1], ys, xs, eps = 10)

  ## exact not approximate
  #nmslib_index = nmslib_create_index(data[big_p1==1], ys, xs)
  #results_nmslib_500[r,:] = nmslib_impute(nmslib_index, data[s500], data[big_p1==1], ys, xs)
  #results_nmslib_1000[r,:] = nmslib_impute(nmslib_index, data[s1000], data[big_p1==1], ys, xs)

  ## nndesc (function is needed)
  NND_index = NNDescent(data[big_p1==1], n_neighbors=50, metric='euclidean')
  NND_index.prepare()
  neighboNND_ids, neighboNND_dists = NND_index.query(data[s500], k=1, epsilon=0.1)
  results_nnd_500[r,:] = np.mean(data[big_p1==1][[i[0] for i in neighboNND_ids]][:, ys], axis = 0)
  neighboNND_ids, neighboNND_dists = NND_index.query(data[s1000], k=1, epsilon=0.1)
  results_nnd_1000[r,:] = np.mean(data[big_p1==1][[i[0] for i in neighboNND_ids]][:, ys], axis = 0)

  ## annoy (function is needed)
  t = AnnoyIndex(len(xs), 'euclidean')  # Length of item vector that will be indexed
  for i in range(data[big_p1==1].shape[0]):
    t.add_item(i, data[big_p1==1][i, xs])

  t.build(50) # 50 trees
  missing_inds = [int(t.get_nns_by_vector(data[s500][i,xs], 1)[0]) for i in range(data[s500].shape[0])]
  results_annoy_500[r,:]=np.mean(data[big_p1==1][missing_inds][:,ys], axis = 0)  
  missing_inds = [int(t.get_nns_by_vector(data[s1000][i,xs], 1)[0]) for i in range(data[s1000].shape[0])]
  results_annoy_1000[r,:]=np.mean(data[big_p1==1][missing_inds][:,ys], axis = 0)  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [44]:
expected = np.stack(
    [np.mean(results_kdtree_500, axis=0),
     np.mean(results_kdtree_app_500, axis=0),
     np.mean(results_nnd_500, axis=0),
     np.mean(results_annoy_500, axis=0),
     np.mean(results_kdtree_1000, axis=0),
     np.mean(results_kdtree_app_1000, axis=0),
     np.mean(results_nnd_1000, axis=0),
     np.mean(results_annoy_1000, axis=0)
     ]
) 

stderrs =  np.stack(
    [np.std(results_kdtree_500, axis=0),
     np.std(results_kdtree_app_500, axis=0),
     np.std(results_nnd_500, axis=0),
     np.std(results_annoy_500, axis=0),
     np.std(results_kdtree_1000, axis=0),
     np.std(results_kdtree_app_1000, axis=0),
     np.std(results_nnd_1000, axis=0),
     np.std(results_annoy_1000, axis=0),
     ]
)

bias = expected - np.mean(data[:,ys], axis=0)
mse = bias**2 + stderrs**2

print(bias)
print(stderrs)
print(mse)

[[-0.00380967 -0.00223866]
 [-0.00412509 -0.0024617 ]
 [ 0.00014134  0.00063521]
 [-0.00381821 -0.00224608]
 [-0.00377138 -0.00182923]
 [-0.00394267 -0.00191611]
 [ 0.0013765   0.0020534 ]
 [-0.00377372 -0.00183135]]
[[0.07452761 0.07339311]
 [0.07472264 0.07323772]
 [0.07342838 0.07226365]
 [0.07452759 0.07339922]
 [0.05295559 0.05292172]
 [0.05284145 0.05300636]
 [0.05202047 0.05075833]
 [0.05294269 0.05291414]]
[[0.00556888 0.00539156]
 [0.00560049 0.00536982]
 [0.00539175 0.00522244]
 [0.00556894 0.00539249]
 [0.00281852 0.00280405]
 [0.00280776 0.00281335]
 [0.00270802 0.00258062]
 [0.00281717 0.00280326]]
